# Notebook 3 – Training NeuLib architectures from scratch

In this notebook we move from using pretrained models to **training a NeuLib architecture from scratch** on ECG data.

Main storyline:

- Configure a GRU-based seq2seq architecture for peak detection.
- Train the model from scratch on a small ECG dataset.
- Evaluate its performance on test segments.
- Continue training the same model for a few extra epochs.

In a later branch of this notebook, we will also:
- Run a small **grid search** over hyperparameters to explore different architectures.

## Step 0 – Setup and data

We first:

- Import NeuLib architectures and utility functions.
- Point to the ECG dataset prepared for peak detection:
  - `X`: input ECG segments
  - `Y`: binary labels or target sequences


In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

import sys

# Add the project directory to sys.path
project_path = os.path.abspath("..")
if project_path not in sys.path:
    sys.path.append(project_path)

from config_ori import DATASETS_ECG_G
import NeuralLib.architectures as archs


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X = os.path.join(DATASETS_ECG_G, 'sample', 'x')
Y = os.path.join(DATASETS_ECG_G, 'sample', 'y_bin')

## Part A – One model from scratch

Before exploring many combinations, we start with **one concrete model**:

1. Inspect which architectures are available in NeuLib.
2. Choose `GRUseq2seq` for ECG peak detection.
3. Define a *single* configuration of hyperparameters.
4. Train, test and continue training this model.


### Available architectures in NeuLib

NeuLib includes several biosignal-oriented architectures:

- GRU encoder–decoder and seq2seq
- Transformer-based encoder–decoder and seq2seq
- Sequence-to-one and sequence-to-sequence variants

For this tutorial we will use **GRUseq2seq**, which is well suited for:

- Peak detection over ECG segments
- Denoising and regression tasks


#### List all the available architectures from Lib

In [3]:
# To check which classes of architectures are available
from NeuralLib.architectures import list_architectures
list_architectures()

['GRUED',
 'GRUseq2one',
 'GRUseq2seq',
 'TransformerED',
 'Transformerseq2one',
 'Transformerseq2seq']

### Step 1 – Define a single GRUseq2seq configuration

To keep the first run simple, we choose:

- A moderate hidden dimension and number of layers.
- Dropout for regularisation.
- A binary **classification** task for peak detection.

This mirrors how pretrained peak-detector models in NeuLib were originally trained,
but here we run a small training loop live.


In [4]:
personal_params = {
    'model_name': 'ECGPeakDetector',
    'n_features': 1,
    'hid_dim': 16,
    'n_layers': 2,
    'dropout': 0.3,
    'learning_rate': 0.01,
    'bidir_per_layer': [True, False], # Example for 2 layers
    'task': 'classification',
    'num_classes': 1,
    }

In [8]:
personal_model = archs.GRUseq2seq(**personal_params)
print(personal_model)

GRUseq2seq(
  (gru_layers): ModuleList(
    (0): GRU(1, 16, batch_first=True, bidirectional=True)
    (1): GRU(32, 16, batch_first=True)
  )
  (dropout_layers): ModuleList(
    (0-1): 2 x Dropout(p=0.3, inplace=False)
  )
  (fc_out): Linear(in_features=16, out_features=1, bias=True)
  (criterion): BCEWithLogitsLoss()
)


### Step 2 – Training parameters

We now specify:

- Number of epochs and batch size (kept small for the demo).
- Dataset name and task description (stored in the training metadata).
- Device configuration (CPU for portability during the tutorial).

The `train_from_scratch` method will use these parameters
to run a full training loop.


In [9]:
# Minimal values for testing purposes
train_params = {
    'path_x': X,
    'path_y': Y,
    'epochs': 3,   
    'batch_size': 1,
    'patience': 2,
    'dataset_name': 'private_gib01',
    'trained_for': 'peak detection',
    'all_samples': False,
    'samples': 3,
    'gpu_id': None,
    'enable_tensorboard': True
}

### Step 3 – Train from scratch

We call:

```python
model_personal.train_from_scratch(**train_params)


In [10]:
final_weight_path = personal_model.train_from_scratch(**train_params)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:751: Checkpoint directory /Users/groupies/Documents/NOVAProjects/NeuralLib-fork/results/ECGPeakDetector/checkpoints/GRUseq2seq_[16, 16]hid_2l_lr0.01_drop[0.3, 0.3]_dt2026-02-18_09-30-19 exists and is not empty.

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | gru_layers     | ModuleList        | 4.2 K  | train
1 | dropout_layers | ModuleList        | 0      | train
2 | fc_out         | Linear            | 17     | train
3 | criterion      | BCEWithLogitsLoss | 0      | train


No GPU available, using CPU.
Checkpoints directory created at /Users/groupies/Documents/NOVAProjects/NeuralLib-fork/results/ECGPeakDetector/checkpoints/GRUseq2seq_[16, 16]hid_2l_lr0.01_drop[0.3, 0.3]_dt2026-02-18_09-30-19
✅ Dataset validation passed.
✅ Dataset validation passed.
TensorBoard logs will be saved to /Users/groupies/Documents/NOVAProjects/NeuralLib-fork/results/ECGPeakDetector/checkpoints/GRUseq2seq_[16, 16]hid_2l_lr0.01_drop[0.3, 0.3]_dt2026-02-18_09-30-19/tensorboard_logs/version_0
Starting training...
Sanity Checking DataLoader 0:   0%|                                                     | 0/2 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|███████████████████████████████████████| 3/3 [00:11<00:00,  0.25it/s, v_num=0, train_loss=0.339]
idation: |                                                                           | 0/? [00:00<?, ?it/s]
idation: |                                                                           | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|███████████████████████| 3/3 [00:12<00:00,  0.25it/s, v_num=0, train_loss=0.143, val_loss=0.254]
idation: |                                                                           | 0/? [00:00<?, ?it/s]
idation: |                                                                           | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████████████████| 3/3 [00:11<00:00,  0.26it/s, v_num=0, train_loss=0.0771, val_loss=0.110]
idation: |                                                                           | 0/? [00:00<?, ?it/s]
idation: |                                                                           | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|█████

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|█████████████████████| 3/3 [00:12<00:00,  0.24it/s, v_num=0, train_loss=0.0771, val_loss=0.0668]
Total training time: 39.05 seconds
{'architecture': 'GRUseq2seq', 'model_name': 'ECGPeakDetector', 'train_dataset': 'private_gib01', 'task': 'peak detection', 'gpu_model': None, 'epochs': 3, 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    decoupled_weight_decay: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    initial_lr: 0.01\n    lr: 0.01\n    maximize: False\n    weight_decay: 1e-05\n)', 'learning_rate': 0.01, 'validation_loss': 0.06678371876478195, 'training_time': 39.050047874450684, 'retraining': False, 'efficiency_flops': None, 'efficiency_params': 4241}
Training complete. Best_model_path: /Users/groupies/Documents/NOVAProjects/NeuralLib-fork/results/ECGPeakDetector/checkpoints/GRUseq2seq_[16, 16]hid_2l_lr0.01_drop[0.3, 0.3]_dt2026-02-18_09-30-19/GRUseq2seq_[16, 16]

### Step 4 – Evaluate on test segments

After training, we can:

- Run `test_on_test_set` to obtain aggregate metrics.
- Use `test_on_single_signal` to get a single prediction
  - Then you could do whatever you want to fit your requirement


In [8]:
predictions, avg_loss = personal_model.test_on_test_set(
    path_x=train_params["path_x"],
    path_y=train_params["path_y"],
    checkpoints_dir=final_weight_path,
    gpu_id=train_params["gpu_id"],
    save_predictions=True,
    all_samples=False,
    samples=5,
)

No GPU available, using CPU.
Using device: cpu
✅ Dataset validation passed.
Weights successfully loaded from /Users/groupies/Documents/NOVAProjects/NeuralLib-fork/results/ECGPeakDetector/checkpoints/GRUseq2seq_[16, 16]hid_2l_lr0.01_drop[0.3, 0.3]_dt2026-02-15_14-20-56/model_weights.pth.
Sample 0: Test Loss: 0.0809
Sample 1: Test Loss: 0.0809
Sample 2: Test Loss: 0.0815
Sample 3: Test Loss: 0.0814
Sample 4: Test Loss: 0.0815
Average Test Loss: 0.0812


In [9]:
single_signal = torch.rand(100, 1)  # Example input signal (sequence length: 100, 1 feature)
signal_prediction = personal_model.test_on_single_signal(single_signal, checkpoints_dir=final_weight_path, gpu_id=train_params["gpu_id"])
print(f"Single Signal Prediction: {signal_prediction}")

No GPU available, using CPU.
Using device: cpu
Weights successfully loaded from /Users/groupies/Documents/NOVAProjects/NeuralLib-fork/results/ECGPeakDetector/checkpoints/GRUseq2seq_[16, 16]hid_2l_lr0.01_drop[0.3, 0.3]_dt2026-02-15_14-20-56/model_weights.pth.
Single Signal Prediction: tensor([[[-1.2131],
         [-1.7901],
         [-2.1081],
         [-2.2918],
         [-2.4027],
         [-2.4823],
         [-2.5259],
         [-2.5597],
         [-2.5826],
         [-2.5985],
         [-2.6169],
         [-2.6283],
         [-2.6363],
         [-2.6426],
         [-2.6442],
         [-2.6469],
         [-2.6461],
         [-2.6467],
         [-2.6454],
         [-2.6521],
         [-2.6545],
         [-2.6577],
         [-2.6549],
         [-2.6564],
         [-2.6607],
         [-2.6596],
         [-2.6603],
         [-2.6626],
         [-2.6623],
         [-2.6652],
         [-2.6670],
         [-2.6671],
         [-2.6672],
         [-2.6657],
         [-2.6655],
         [-2.66

### Step 5 – Continue training the same model

Instead of starting from scratch again, we can:

- Reload the best checkpoint, or
- Directly call a **retraining** / **continue_training** method on the same model.

Typical use cases:

- Extending training for a few extra epochs.
- Adapting the model after we collect more data from the same domain.

The next cell shows how to run a short extra training phase
and observe whether the validation loss still improves.


In [11]:
# Example: reusing train_params with a small number of extra epochs
# retrain the model using the best found parameters
train_params_retrain = train_params.copy()
train_params_retrain['epochs'] = 4
personal_model.retrain(
    checkpoints_directory=final_weight_path, # checkpoints directory where the models weights and parameters were stored in the previous step
    path_x=train_params_retrain['path_x'],
    path_y=train_params_retrain['path_y'],
    patience=train_params_retrain['patience'],
    batch_size=train_params_retrain['batch_size'],
    epochs=train_params_retrain['epochs'],
    gpu_id=train_params_retrain['gpu_id'],
    all_samples=train_params_retrain['all_samples'],
    samples=train_params_retrain['samples'],
    dataset_name=train_params_retrain['dataset_name'],
    trained_for=train_params_retrain['trained_for'],
    enable_tensorboard=train_params_retrain['enable_tensorboard'],
)


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:751: Checkpoint directory /Users/groupies/Documents/NOVAProjects/NeuralLib-fork/results/ECGPeakDetector/checkpoints/GRUseq2seq_[16, 16]hid_2l_lr0.01_drop[0.3, 0.3]_retraining_dt2026-02-15_14-21-31 exists and is not empty.

  | Name           | Type              | Params | Mode
------------------------------------------------------------
0 | gru_layers     | ModuleList        | 4.2 K  | eval
1 | dropout_layers | ModuleList        | 0      | eval
2 | fc_out         | Linear            | 17     | eval
3 | criterion      | BCEWithLogitsLoss | 0      | 

No GPU available, using CPU.
Found existing .pth file: /Users/groupies/Documents/NOVAProjects/NeuralLib-fork/results/ECGPeakDetector/checkpoints/GRUseq2seq_[16, 16]hid_2l_lr0.01_drop[0.3, 0.3]_dt2026-02-15_14-20-56/model_weights.pth
Weights loaded successfully from /Users/groupies/Documents/NOVAProjects/NeuralLib-fork/results/ECGPeakDetector/checkpoints/GRUseq2seq_[16, 16]hid_2l_lr0.01_drop[0.3, 0.3]_dt2026-02-15_14-20-56/model_weights.pth
✅ Dataset validation passed.
✅ Dataset validation passed.
TensorBoard logs will be saved to /Users/groupies/Documents/NOVAProjects/NeuralLib-fork/results/ECGPeakDetector/checkpoints/GRUseq2seq_[16, 16]hid_2l_lr0.01_drop[0.3, 0.3]_retraining_dt2026-02-15_14-21-31/tensorboard_logs/version_0
Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:527: Found 9 module(s) in eval mode at the start of training. This may lead to unexpected behavior during training. If this is intentional, you can ignore this warning.


Epoch 3: 100%|██████████| 3/3 [00:10<00:00,  0.28it/s, v_num=0, train_loss=0.0312, val_loss=0.0301]

`Trainer.fit` stopped: `max_epochs=4` reached.


Epoch 3: 100%|██████████| 3/3 [00:10<00:00,  0.28it/s, v_num=0, train_loss=0.0312, val_loss=0.0301]
Total training time: 44.12 seconds
{'architecture': 'GRUseq2seq', 'model_name': 'ECGPeakDetector', 'train_dataset': 'private_gib01', 'task': 'peak detection', 'gpu_model': None, 'epochs': 4, 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    decoupled_weight_decay: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    initial_lr: 0.01\n    lr: 0.01\n    maximize: False\n    weight_decay: 1e-05\n)', 'learning_rate': 0.01, 'validation_loss': 0.030109049752354622, 'training_time': 44.120843172073364, 'retraining': True, 'efficiency_flops': None, 'efficiency_params': 4241, 'training_history': {'architecture': 'GRUseq2seq', 'model_name': 'ECGPeakDetector', 'train_dataset': 'private_gib01', 'task': 'peak detection', 'gpu_model': None, 'epochs': 3, 'optimizer': 'Adam (\nParameter Group 0\n    amsgr

## Part B – Exploring architectures with grid search

Once we have seen one model trained end-to-end,
we can explore a small hyperparameter space:

- Different hidden dimensions and numbers of layers.
- Alternative dropout values.
- Potentially different bidirectionality patterns.

NeuLib provides a simple `grid_search` utility that:

- Iterates over combinations of architecture hyperparameters.
- Trains a model for each combination (with limited epochs in this demo).
- Logs validation loss and basic efficiency statistics.


In [12]:
architecture_name = 'GRUseq2seq'
archs.validate_architecture_name(architecture_name)

archi_params_options = {
    "model_name": "ECGPeakDetector",
    "n_features": [1],
    "hid_dim": [[32, 64, 64], [64, 64, 64], [64, 128, 64], [64, 128]],
    "n_layers": [3, 2],
    "dropout": [0.3, 0],
    "learning_rate": [0.001],
    "bidirectional": [True],
    "task": ["classification"],
    "num_classes": [1],
}

Architecture GRUseq2seq is valid.


In [13]:
train_params = {
    'path_x': X,
    'path_y': Y,
    'epochs': 2,
    'batch_size': 1,
    'patience': 2,
    'dataset_name': 'private_test',
    'trained_for': 'peak detection',
    'all_samples': False,
    'samples': 3,
    'gpu_id': None,
    'enable_tensorboard': True,
}

#### Step 2: Perform grid search

In [ ]:
best_dir, best_val_loss, val_losses = archs.run_grid_search(architecture_name, archi_params_options, train_params)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:751: Checkpoint directory /Users/groupies/Documents/NOVAProjects/NeuralLib-fork/results/ECGPeakDetector/checkpoints/GRUseq2seq_[32, 64, 64]hid_3l_bidirTrue_lr0.001_drop[0.3, 0.3, 0.3]_dt2026-02-15_14-22-15 exists and is not empty.

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | gru_layers     | ModuleList        | 131 K  | train
1 | dropout_layers | ModuleList        | 0      | train
2 | fc_out         | Linear            | 129    | train
3 | criterion      | BCEWithLogitsLoss | 0      | train
-------------------------------------------------------------
131 K     Trainable params
0         Non-trainable params
131 K     Total params
0.525     Total estimated model params 

Architecture GRUseq2seq is valid.
Running grid search with 16 combinations.

Training model 1/16 with parameters: {'model_name': 'ECGPeakDetector', 'n_features': 1, 'hid_dim': [32, 64, 64], 'n_layers': 3, 'dropout': 0.3, 'learning_rate': 0.001, 'bidirectional': True, 'task': 'classification', 'num_classes': 1}
Architecture GRUseq2seq is valid.
No GPU available, using CPU.
Checkpoints directory created at /Users/groupies/Documents/NOVAProjects/NeuralLib-fork/results/ECGPeakDetector/checkpoints/GRUseq2seq_[32, 64, 64]hid_3l_bidirTrue_lr0.001_drop[0.3, 0.3, 0.3]_dt2026-02-15_14-22-15
✅ Dataset validation passed.
✅ Dataset validation passed.
TensorBoard logs will be saved to /Users/groupies/Documents/NOVAProjects/NeuralLib-fork/results/ECGPeakDetector/checkpoints/GRUseq2seq_[32, 64, 64]hid_3l_bidirTrue_lr0.001_drop[0.3, 0.3, 0.3]_dt2026-02-15_14-22-15/tensorboard_logs/version_0
Starting training...
Epoch 0:   0%|          | 0/3 [00:00<?, ?it/s]                             

### Step 3: Test the Best model performance on your test set

In [ ]:
# Load architecture parameters from the best checkpoints directory
architecture_params = archs.get_hparams_from_checkpoints(best_dir)
# Initialize the model with the best architecture parameters
model = archs.GRUseq2seq(**architecture_params)

predictions, avg_loss = model.test_on_test_set(
    path_x=train_params["path_x"],
    path_y=train_params["path_y"],
    checkpoints_dir=best_dir,
    gpu_id=train_params["gpu_id"],
    save_predictions=True,
    all_samples=False,
    samples=5,
)

## Try other architectures to fit your tasks
You can change `architecture_name` to any of the entries listed above and re-run the cells that instantiate, train and evaluate the model.
